In [37]:
import pandas as pd

# Veriyi yükleyelim
file_path = '/content/drug_train.csv'
df = pd.read_csv(file_path , sep=";" , on_bad_lines='skip')
df = df.drop(columns=['Unnamed: 2'])
df = df.drop(columns=['Unnamed: 3'])

<ipython-input-37-5f1f712614ef>:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path , sep=";" , on_bad_lines='skip')


In [38]:
df['condition'] = df['condition'].astype(str)
df['drugName'] = df['drugName'].astype(str)

In [39]:
df.head()

,condition,drugName
0,Bipolar Disorde,Aripiprazole
1,Depression,L-methylfolate
2,Bipolar Disorde,Lamotrigine
3,Chronic Myelogenous Leukemia,Nilotinib
4,Insomnia,Trazodone


In [5]:
!pip install transformers
!pip install torch

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [41]:
# BERT için tokenizer ve model yükleyelim
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['drugName'].unique()))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
# Veriyi tokenleştirelim
def tokenize_function(examples):
    return tokenizer(examples['condition'], padding="max_length", truncation=True)

In [43]:
# Veriyi train ve test olarak ayıralım
train_df, test_df = train_test_split(df, test_size=0.2)

In [44]:
# Tokenize edilmiş veri setlerini oluşturalım
train_encodings = tokenizer(train_df['condition'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df['condition'].tolist(), truncation=True, padding=True)

In [45]:
class DrugDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [46]:
train_labels = train_df['drugName'].astype('category').cat.codes.values
test_labels = test_df['drugName'].astype('category').cat.codes.values

train_labels = torch.tensor(train_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)


In [47]:
train_dataset = DrugDataset(train_encodings, train_labels)
test_dataset = DrugDataset(test_encodings, test_labels)

In [14]:
!pip install transformers[torch] --upgrade
!pip install accelerate --upgrade

In [48]:
from transformers import Trainer, TrainingArguments

In [52]:
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}


In [68]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=2e-5
    # evaluate at the end of each epoch
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [69]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # evaluation metrics
)

In [70]:
trainer.train()


<ipython-input-45-923d77eca510>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss,Accuracy
1,2.969200,11.712157,0.000000
2,3.190900,11.817636,0.000000
3,3.159300,11.848819,0.000000


<ipython-input-45-923d77eca510>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-45-923d77eca510>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-45-923d77eca510>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-45-923d77eca510>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone

TrainOutput(global_step=24231, training_loss=3.1449220828731357, metrics={'train_runtime': 3232.2326, 'train_samples_per_second': 119.938, 'train_steps_per_second': 7.497, 'total_flos': 4727425294687860.0, 'train_loss': 3.1449220828731357, 'epoch': 3.0})

In [58]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [71]:
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [72]:
# Modeli test edelim
def predict(condition):
    inputs = tokenizer(condition, return_tensors="pt", truncation=True, padding=True).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    predicted_label = predictions.cpu().numpy()[0]
    return df['drugName'].astype('category').cat.categories[predicted_label]



In [82]:
condition = "I have Insomnia"
print(predict(condition))

Ursodiol


In [79]:
model.save_pretrained("./final")